In [5]:
import requests
import pandas as pd
import os
from datetime import datetime

# **Récupérer les données temps réel CTA**

In [6]:
CTA_API_KEY = "qivkxUrXhq9fuX6m9UyCxSUep"  # remplace par ta clé
CTA_BASE_URL = "https://www.ctabustracker.com/bustime/api/v3"

### Récupérer la liste des routes

In [ ]:
all_stops = []

for route in df_routes["rt"]:
    df_stops = get_cta_stops(route)
    if not df_stops.empty:
        df_stops["route"] = route
        all_stops.append(df_stops)

df_all_stops = pd.concat(all_stops, ignore_index=True)
df_all_stops.to_csv(f"{OUTPUT_DIR}/cta_stops.csv", index=False)

print("✅ Arrêts CTA sauvegardés")


In [4]:
# Récupérer la liste des routes
def get_cta_routes(api_key):
    params = {"key": api_key, "format": "json"}
    response = requests.get(f"{CTA_BASE_URL}/getroutes", params=params)
    data = response.json().get("bustime-response", {}).get("routes", [])
    return pd.DataFrame(data)

# Test
df_routes = get_cta_routes(CTA_API_KEY)
print(df_routes.head())

   rt                       rtnm    rtclr rtdd
0   1  Bronzeville/Union Station  #336633    1
1   2          Hyde Park Express  #993366    2
2   3                 King Drive  #009900    3
3   4              Cottage Grove  #cc3300    4
4  X4      Cottage Grove Express  #006666   X4


### Récupérer la liste des arrêts par route

In [12]:
# Récupérer la liste des arrets par route
def get_cta_stops(api_key, route):
    params = {"key": api_key, "format": "json", "rt": route }
    response = requests.get(f"{CTA_BASE_URL}/getstops", params=params)
    data = response.json().get("bustime-response", {}).get("stops", [])
    return pd.DataFrame(data)

# Test
df_stops = get_cta_stops(CTA_API_KEY,"2")
print(df_stops.head())

Empty DataFrame
Columns: []
Index: []


In [ ]:
# Dossier où seront sauvegardés les fichiers
OUTPUT_DIR = "../data/raw"
os.makedirs(OUTPUT_DIR, exist_ok=True)


# Sauvegarde un DataFrame CSV avec timestamp
def save_df(df, prefix):
    timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
    filename = f"{prefix}_{timestamp}.csv"
    filepath = os.path.join(OUTPUT_DIR, filename)
    df.to_csv(filepath, index=False)
    print(f"✅ Sauvegardé : {filepath}")